# Support Vector Machines

In this notebook we introduce Support Vector Machines (SVM) from a theoretical perspective.
We willl carefully motivate and formulate the mathematical model behind SVMs,
and will show you how to train an SVM using a generic convex optimization solver.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors
import numpy as np
import scipy.stats
from IPython.display import Image

## Introduction

Let's start by considering a simple binary classification problem.
The set-up is as follows: we have a dataset consisting of two features
denoted $X_1$ and $X_2$, and a label denoted by the variable
$Y$. We'll denote by
$\mathbf{x}_1$, $\mathbf{x}_2$, ..., $\mathbf{x}_n$ a set
of samples from the distribution of $(X_1, X_2)$, and will denote
their labels by $y_1$, $y_2$, ..., $y_n$. For reasons that will
become apparent later, the labels will be assumed to be taking
values in $\{-1, 1\}$; the *positive class* will refer to the data points
with label $1$, and the *negative class* will refer to the data points
with label $-1$. We'll use $S_+$ to denote the set of samples with label
$1$ and $S_-$ to denote the set of samples with label $-1$.

In the first part of this tutorial, our toy dataset will consist of
20 samples generated from a 2D normal random
variable with mean vector $(1, 0)$ and covariance matrix
$$\begin{pmatrix}
0.05 & 0 \\
0 & 0.05
\end{pmatrix},$$
for the positive class, and the negative class will consist of
20 samples generated from a 2D normal random variable with mean
vector $(0, 1)$ and the same covariance matrix as for the positive class.

In [ ]:
np.random.seed(42)

In [ ]:
n_samples_per_class = 20

In [ ]:
Y_plus_dist = scipy.stats.multivariate_normal(
    mean=np.array([1, 0]), cov=np.array([[0.05, 0], [0, 0.05]])
)

Y_plus_sample = Y_plus_dist.rvs(n_samples_per_class)

Y_minus_dist = scipy.stats.multivariate_normal(
    mean=np.array([0, 1]), cov=np.array([[0.05, 0], [0, 0.05]])
)

Y_minus_sample = Y_minus_dist.rvs(n_samples_per_class)

features = np.concatenate([Y_plus_sample, Y_minus_sample], axis=0)

labels = np.ones(shape=(2 * n_samples_per_class, 1))
labels[n_samples_per_class:] = -1

data = np.concatenate([features, labels], axis=1)

In [ ]:
colors = ["red", "green"]

In [ ]:
plt.scatter(x=data[:20, 0], y=data[:20, 1], c="green", label="$Y = 1$")
plt.scatter(x=data[20:, 0], y=data[20:, 1], c="red", label="$Y = -1$")
plt.axis("equal")
plt.xlabel("$X_1$")
plt.ylabel("$X_2$")
plt.xlim((-0.5, 1.5))
plt.ylim((-0.5, 1.5))
plt.legend()
plt.show()

It seems like one should be able to separate this dataset *linearly*,
i.e. that one should be able to find a straight line which separates the
two classes perfectly. In mathematical terms, this means
that there should exist coefficients $w_1$, $w_2$ and $b$
such that $Y = 1$ if $w_1X_1 + w_2X_2 + b > 0$ and
$Y = -1$ if $w_1X_1 + w_2X_2 + b < 0$. Let's have a go at
drawing a few possible separating lines.

In [ ]:
X_1 = np.linspace(start=-2, stop=2, num=50)

In [ ]:
line1 = matplotlib.lines.Line2D(
    xdata=X_1,
    ydata=X_1,
    label="$X_2 = X_1$",
    color="#9b9b9b",
    linestyle="solid",
)
line2 = matplotlib.lines.Line2D(
    xdata=X_1,
    ydata=2 * X_1 + 0.3,
    label="$X_2 = 2X_1$+0.3",
    color="#9b9b9b",
    linestyle="dashed",
)
line3 = matplotlib.lines.Line2D(
    xdata=X_1,
    ydata=0.1 * X_1 + 0.5,
    label="$X_2 = 0.1X_1$+0.5",
    color="#9b9b9b",
    linestyle="dotted",
)

In [ ]:
fig, ax = plt.subplots()
plt.scatter(x=data[:20, 0], y=data[:20, 1], c="green", label="$Y = 1$")
plt.scatter(x=data[20:, 0], y=data[20:, 1], c="red", label="$Y = -1$")
plt.axis("equal")
plt.xlabel("$X_1$")
plt.ylabel("$X_2$")
plt.xlim((-0.5, 1.5))
plt.ylim((-0.5, 1.5))

ax.add_line(line1)
ax.add_line(line2)
ax.add_line(line3)
plt.legend(bbox_to_anchor=(1, 1), handles=[line1, line2, line3])
plt.show()

Each one of these three lines separates the data.
However, we are intuitively drawn to choose the solid line over the other
two as a separator of the classes. This is because we expect it
to generalize better on unseen data: the dashed and dotted lines both
seem to overfit to the boundary of the red class. Let us generate
a few more datapoints to verify this assumption on this particular dataset.

In [ ]:
n_extra_samples_per_class = 200

In [ ]:
Y_plus_extra_sample = Y_plus_dist.rvs(n_extra_samples_per_class)

Y_minus_extra_sample = Y_minus_dist.rvs(n_extra_samples_per_class)

features_extra = np.concatenate(
    [Y_plus_extra_sample, Y_minus_extra_sample], axis=0
)

labels_extra = np.ones(shape=(2 * n_extra_samples_per_class, 1))
labels_extra[n_extra_samples_per_class:] = -1

data_extra = np.concatenate([features_extra, labels_extra], axis=1)

In [ ]:
line1 = matplotlib.lines.Line2D(
    xdata=X_1,
    ydata=X_1,
    label="$X_2 = X_1$",
    color="#9b9b9b",
    linestyle="solid",
)
line2 = matplotlib.lines.Line2D(
    xdata=X_1,
    ydata=2 * X_1 + 0.3,
    label="$X_2 = 2X_1$+0.3",
    color="#9b9b9b",
    linestyle="dashed",
)
line3 = matplotlib.lines.Line2D(
    xdata=X_1,
    ydata=0.1 * X_1 + 0.5,
    label="$X_2 = 0.1X_1$+0.5",
    color="#9b9b9b",
    linestyle="dotted",
)

fig, ax = plt.subplots()
plt.scatter(
    x=data_extra[:n_extra_samples_per_class, 0],
    y=data_extra[:n_extra_samples_per_class, 1],
    c="green",
    label="$Y = 1$",
)
plt.scatter(
    x=data_extra[n_extra_samples_per_class:, 0],
    y=data_extra[n_extra_samples_per_class:, 1],
    c="red",
    label="$Y = -1$",
)

plt.axis("equal")
plt.xlabel("$X_1$")
plt.ylabel("$X_2$")
plt.xlim((-0.5, 1.5))
plt.ylim((-0.5, 1.5))
ax.add_line(line1)
ax.add_line(line2)
ax.add_line(line3)
plt.legend(bbox_to_anchor=(1, 1), handles=[line1, line2, line3])
plt.show()

It seems like our intuition was correct! After resampling $200$ data
points from both distributions, the solid line is the only one to
keep a perfect accuracy score. A Support Vector Machine
(also known as a Support Vector Classifier, if we wish to emphasise that it
is being used for classification purposes)
attempts to find the line that separates the data in the best possible way,
just as we did. This assumes an appropriate definition for what
a "good separation" means. In the case of SVMs, the best separating line
is the one which is as "far away from the boundary of either class as possible".
The dashed and dotted lines, being "too close" to the boundary of the classes,
are not desirable. In the rest of this post, we'll strive to translate this
observation into mathematical terms. It's easy to get lost in the details
of the algorithm, but this should not make you forget that SVMs, at their
heart, are doing nothing more than finding the line whose distance to
either class is as large as possible.

### Hyperplanes

Whilst our introductory example dealt with a binary classification problem with two predictors,
real-life problems will typically feature many more predictors. Hence, from now on, let's assume
that we have $k$ variables $X_1$, $X_2$, ..., $X_k$ available to us in order to classify the data into two classes.
(Rest assured that our toy examples will still have only 2 features, so as to make visualisation as easy as possible!).
In higher dimensions, it no longer makes no sense to speak of a "straight line" separating the data.
The mathematical objects which generalize 2D lines in higher dimensional space 
are called a *hyperplanes*. A hyperplane in $k$-dimensional space is the surface defined by the equation
$$w_0 X_0 + w_1 X_1 + \cdots + w_{k-1}X_{k-1} + b = 0.$$
You'll notice that when $k=2$ this is the straight line in 2D space that we know and love,
and when $k=3$ this is simply a plane in 3-dimensional space. This equation can be
written in terms of scalar products of vectors: if $\mathbf{x} = (X_1, X_2, \ldots, X_k)$
then this is equivalent to
$$\mathbf{w}\cdot \mathbf{x} + b = 0.$$
From now on, we'll use $\Pi$ to denote the surface defined by the above equation.
There is a natural interpretation for the coefficients of a hyperplane: 
$$\mathbf{w} = (w_0, w_1, \ldots, w_{k-1})$$
is a vector which is *normal* (i.e. perpendicular) to $\Pi$, and
$$b = - \mathbf{w}\cdot\mathbf{v_0},$$
where $\cdot$ denotes the scalar product in $k$-dimensional space
and $\mathbf{v_0}$ is any vector belonging to $\Pi$.

In [ ]:
Image("img/fig1.png")

### Distances

If we are to compute the hyperplane which is "furthest away from the class boundaries", we had better define
what we mean by the distance between a set of data points and a hyperplane!
The *distance* $d(\mathbf{x}, \Pi)$ between a data point $\mathbf{x}$ and a hyperplane $\Pi$
is defined to be the $l^2$-norm of $\mathbf{x} - \mathbf{x_\perp}$ where $\mathbf{x_\perp}$ is the orthogonal
projection of $\mathbf{x}$ onto $\Pi$. After some algebra, we have:
$$d(\mathbf{x}, \Pi)^2 = \frac{(\mathbf{w}\cdot\mathbf{x} + b)^2}{\|\mathbf{w}\|^2}.$$
This is consistent with the equation defining $\Pi$: the set of
vectors $\mathbf{x}$ belonging to $\Pi$ is those for which $d(\mathbf{x}, \Pi) = 0$.

Then, the distance between a set of vectors $S$ and $\Pi$ is defined to be the distance between $\Pi$ and the data point
in $S$ which is closest to $\Pi$, i.e.
$$d(S, \Pi) = \min_{\mathbf{x} \in S} d(\mathbf{x}, \Pi).$$

We now have a mathematically sound definition of *distance* which a computer algorithm
will be able to optimize.

In [ ]:
Image("img/fig2.png")

### The linearly separable case
As we mentioned in the introduction, a support vector classifier aims to find the hyperplane $\Pi$ which
separates the two classes whilst maximizing the distance between the hyperplane and the boundary of either class.
In mathematical terms, separating the data means finding $\mathbf{w}$ and $b$ such that
$$\mathbf{w}\cdot \mathbf{x} + b > 0$$
for members of the positive class, and
$$\mathbf{w}\cdot \mathbf{x} + b < 0$$
for members of the negative class. To require the hyperplane to
be as far away from the boundary of either class means finding $\mathbf{w}$ and $b$ such that
$$\min\{ d(S_+, \Pi), d(S_-, \Pi) \}$$
is as large as possible. It is not be too hard to convince oneself
that for the optimal plane $\Pi$ it must be the case that $d(S_+, \Pi) = d(S_-, \Pi)$. (Recall that $S_+$ and $S_-$ denote the vectors with labels $1$ and $-1$, respectively.) Indeed, if this were not the case, then we could always reduce the imbalance between the two distances by translating the plane along $\mathbf{w}$ in the appropriate direction. This would then increase $\min\{ d(S_+, \Pi), d(S_-, \Pi) \}$, a contradiction. This has the consequence that if $\mathbf{x_+} \in S_+$ is closest to $\Pi$ and $\mathbf{x_-} \in S_-$ is closest to $\Pi$ then
$$\mathbf{w}\cdot\mathbf{x_+} + b = -(\mathbf{w}\cdot\mathbf{x_-} + b).$$
By multiplying the equation $\mathbf{w}\cdot\mathbf{x} + b = 0$ by a positive constant if needed, we may assume that $\mathbf{w}\cdot\mathbf{x_+} + b = 1$ for the vectors
in $S_+$ which are closest to $\Pi$ and $\mathbf{w}\cdot\mathbf{x_-} + b = -1$ for the vectors in $S_-$ which are closest
to $\Pi$. The square of the distance between $\Pi$ and these vectors, and hence between $\Pi$ and either one of the two classes, is then simply $1 / \|w\|^2$. Accordingly, we may restate the support vector classification problem as follows: Find $\mathbf{w}$ and $b$ so that $\|\mathbf{w}\|^2$ is as small as possible subject to $\mathbf{w} \cdot \mathbf{x} + b \geq 1$  for each  $\mathbf{x} \in S_+$, and
$\mathbf{w} \cdot \mathbf{x} + b \leq -1$  for each  $\mathbf{x} \in S_-$. Since we cleverly chose our labels to be $+1$ for members of the positive class and $-1$ for members of the negative class, we may simplify this further into

**Support Vector Machine, Hard Margin**: Minimize $$\frac{1}{2}\|\mathbf{w}\|^2$$
subject to
$$y_i\left(\mathbf{w} \cdot \mathbf{x}_i + b\right) -1 \geq 0 \text{ for }1\leq i\leq n.$$
where $y_i$ is the $i$-th label.

(It will be clearer in the next section why we called this the "hard margin" version of the SVM optimization problem.)
This is a *convex* quadratic optimization problem. It is not necessary to know exactly what this means; but
convexity makes the optimization problem easier to solve since it implies that a local minimum must also be a global minimum.
Note that the factor of $1/2$ is included for convenience.
In what follows we'll use the `cvxopt` package to train an SVM
on our synthetically generated dataset.

In [ ]:
!pip install cvxopt

In [ ]:
from cvxopt.solvers import qp
from cvxopt import matrix

The quadratic programming solver from the `cvxopt` package expects the quadratic problem
to be posed in a slightly different form; see http://cvxopt.org/userguide/coneprog.html#quadratic-programming
for more details.

In [ ]:
P = matrix(np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 0.0]]))

In [ ]:
q = matrix(np.array([0.0, 0.0, 0.0]).T)

In [ ]:
G = matrix(
    -data[:, 2].reshape((-1, 1))
    * np.concatenate([data[:, :2], np.ones((data.shape[0], 1))], axis=1)
)

In [ ]:
h = matrix(-np.ones(shape=(data.shape[0], 1)))

In [ ]:
solution = qp(P, q, G, h)

In [ ]:
coeffs = np.array(solution["x"])

In [ ]:
print(coeffs)

The equation of the separating hyperplane found by the SVM algorithm is $2.28X_1 - 1.06X_2-0.53 = 0$.
Let's find the coordinates of the vectors that are closest to this hyperplane. These vectors are called the
*support vectors* of the separating hyperplane. They have the important property that removing all the
other elements from $S_+$ and $S_-$ would leave the optimization problem unchanged in that the
optimal separating hyperplane would still remain the same. 

In [ ]:
tol = 0.0001

In [ ]:
xplus = data[
    (np.abs(np.matmul(data[:, :2], coeffs[:2]) + coeffs[2] - 1) < tol).reshape(
        -1
    ),
    :2,
]

In [ ]:
xminus = data[
    (np.abs(np.matmul(data[:, :2], coeffs[:2]) + coeffs[2] + 1) < tol).reshape(
        -1
    ),
    :2,
]

Now let's plot our findings!

In [ ]:
linesvc = matplotlib.lines.Line2D(
    xdata=X_1,
    ydata=-coeffs[0] / coeffs[1] * X_1 - coeffs[2] / coeffs[1],
    label="$\mathbf{w}\cdot\mathbf{x} + b = 0$",
    color="#e88127",
    linestyle="solid",
)

In [ ]:
linesvcplus = matplotlib.lines.Line2D(
    xdata=X_1,
    ydata=-coeffs[0] / coeffs[1] * X_1 - coeffs[2] / coeffs[1] - 1,
    label="$\mathbf{w}\cdot\mathbf{x} + b = 1$",
    color="#9b9b9b",
    linestyle="dotted",
)

In [ ]:
linesvcminus = matplotlib.lines.Line2D(
    xdata=X_1,
    ydata=-coeffs[0] / coeffs[1] * X_1 - coeffs[2] / coeffs[1] + 1,
    label="$\mathbf{w}\cdot\mathbf{x} + b = -1$",
    color="#9b9b9b",
    linestyle="dashed",
)

In [ ]:
fig, ax = plt.subplots()

plt.axis("equal")
plt.scatter(
    x=data[:n_samples_per_class, 0],
    y=data[:n_samples_per_class, 1],
    c="green",
    label="$Y = 1$",
)
plt.scatter(
    x=data[n_samples_per_class:, 0],
    y=data[n_samples_per_class:, 1],
    c="red",
    label="$Y = -1$",
)
plt.axis("equal")
plt.xlabel("$X_1$")
plt.ylabel("$X_2$")
plt.xlim((-0.5, 1.5))
plt.ylim((-0.5, 1.5))

sv = plt.scatter(
    x=np.concatenate([xplus[:, 0], xminus[:, 0]]),
    y=np.concatenate([xplus[:, 1], xminus[:, 1]]),
    marker="*",
    s=250,
    label="Support Vectors",
)
ax.add_line(linesvc)
ax.add_line(linesvcplus)
ax.add_line(linesvcminus)

plt.legend(
    bbox_to_anchor=(1, 1), handles=[linesvc, sv, linesvcplus, linesvcminus]
)
plt.show()

The region between the planes defined by $\mathbf{w}\cdot\mathbf{x} + b = -1$ and $\mathbf{w}\cdot\mathbf{x} + b = 1$ is called the *margin*.
The *width* of the margin is $2/\| \mathbf{w} \|$; this is the distance between the two planes defining the margin.

### The non-separable case

So far we (implicitly) assumed that the training dataset was linearly separable.
This need not be the case, and it certainly is not for the two classes
of vectors plotted in red and green below.

In [ ]:
Y_plus_insep_dist = scipy.stats.multivariate_normal(
    mean=np.array([1, 0]), cov=np.array([[0.5, 0], [0, 0.5]])
)

Y_plus_insep_sample = Y_plus_insep_dist.rvs(n_samples_per_class)

Y_minus_insep_dist = scipy.stats.multivariate_normal(
    mean=np.array([0, 1]), cov=np.array([[0.5, 0], [0, 0.5]])
)

Y_minus_insep_sample = Y_minus_insep_dist.rvs(n_samples_per_class)

features_insep = np.concatenate(
    [Y_plus_insep_sample, Y_minus_insep_sample], axis=0
)

labels_insep = np.ones(shape=(2 * n_samples_per_class, 1))
labels_insep[n_samples_per_class:] = -1

data_insep = np.concatenate([features_insep, labels_insep], axis=1)

In [ ]:
plt.scatter(
    x=data_insep[:20, 0], y=data_insep[:20, 1], c="green", label="$Y = 1$"
)
plt.scatter(
    x=data_insep[20:, 0], y=data_insep[20:, 1], c="red", label="$Y = -1$"
)
plt.axis("equal")
plt.xlabel("$X_1$")
plt.ylabel("$X_2$")
plt.xlim((-0.5, 1.5))
plt.ylim((-0.5, 1.5))
plt.legend()
plt.show()

Thankfully, this is not the end for Support Vector Machines.
Instead of enforcing a *hard* margin by requiring $y_i\left(\mathbf{w} \cdot \mathbf{x}_i + b\right) -1 \geq 0$
for each member of our dataset, let  us allow this inequality to be violated by some of them,
but let us make sure that we penalize the algorithm whenever it decides to do this. This is
called a *soft* margin.

**Support Vector Machine, Soft Margin**: Minimize $$\frac{1}{2}\|\mathbf{w}\|^2 + C\left(\sum_{i=1}^n \xi_i\right)^k$$
subject to
$$\xi_i \geq 0 \text{ for } 1 \leq i \leq n \text{, and}$$
$$y_i\left(\mathbf{w} \cdot \mathbf{x}_i + b\right) -1 \geq -\xi_i \text{ for } 1 \leq i \leq n.$$
where $y_i$ is the $i$-th label.

Here, $C>0$ and $k\geq 1$ are constant hyperparameters. Typically one chooses $k=1$ as it simplifies the optimization problem (more on this below),
and $C$ is tuned through cross-validation. Choosing a smaller value of $C$ makes the margin softer: we are more tolerant to violations
of the margin. The variables $\xi_i$ are called *slack variables*. When our algorithm chooses to assign a non-zero positive value to a slack
variable, the margin is violated by the corresponding data point, and it becomes a support vector. 

Note that in practice this is always the form in which support vector machines are trained. Not only does this allow us to deal with the
linearly inseparable case, but it can also be beneficial even when the dataset is linearly separable! The reason for this is that
we are allowing the algorithm to look for a larger margin provided the incurred penalty is not too large. This means that
we are overfitting less to the particular values of the vectors closest to the separating hyperplane.

There is a good theoretical reason why the optimal separating plane for our initial linearly separable toy example is defined 
by the equation $X_2 = X_1$ (exercise for the reader!). However, the line we previously found had a slope roughly equal to 2.
Can the soft margin algorithm come to the rescue? Let's try it out with $k=1$ and $C=0.1$.

In [ ]:
C = 0.1

In [ ]:
P = np.zeros(shape=(2 * n_samples_per_class + 3, 2 * n_samples_per_class + 3))
P[0, 0] = 1
P[1, 1] = 1
P = matrix(P)

In [ ]:
q = C * np.ones(shape=(2 * n_samples_per_class + 3, 1))
q[0] = 0.0
q[1] = 0.0
q[2] = 0.0
q = matrix(q)

In [ ]:
G = -data[:, 2].reshape((-1, 1)) * np.concatenate(
    [data[:, :2], np.ones((data.shape[0], 1))], axis=1
)
G = np.concatenate([G, -np.identity(2 * n_samples_per_class)], axis=1)
G_positive_penalty = np.concatenate(
    [
        np.zeros(shape=(2 * n_samples_per_class, 3)),
        -np.identity(2 * n_samples_per_class),
    ],
    axis=1,
)
G = np.concatenate([G, G_positive_penalty], axis=0)
G = matrix(G)

In [ ]:
h = np.concatenate(
    [-np.ones(shape=(data.shape[0], 1)), np.zeros(shape=(data.shape[0], 1))],
    axis=0,
)
h = matrix(h)

In [ ]:
solution = qp(P, q, G, h)

In [ ]:
coeffs = np.array(solution["x"])

In [ ]:
coeffs[:3]

The separating hyperplane is now determined by the equation $0.91X_1 - 0.9X_2 + 0.01$, i.e. $X_2 = 1.008X_1 -0.019$.
This is much closer to the optimal separating line, defined by $X_2 = X_1$, than the separating line we obtained by
enforcing a hard margin!

In [ ]:
xsupport = data[
    (np.abs(np.matmul(data[:, :2], coeffs[:2]) + coeffs[2]) < 1).reshape(-1),
    :2,
]

In [ ]:
linesvc = matplotlib.lines.Line2D(
    xdata=X_1,
    ydata=-coeffs[0] / coeffs[1] * X_1 - coeffs[2] / coeffs[1],
    label="$\mathbf{w}\cdot\mathbf{x} + b = 0$",
    color="#e88127",
    linestyle="solid",
)

In [ ]:
linesvcplus = matplotlib.lines.Line2D(
    xdata=X_1,
    ydata=-coeffs[0] / coeffs[1] * X_1 - coeffs[2] / coeffs[1] - 1,
    label="$\mathbf{w}\cdot\mathbf{x} + b = 1$",
    color="#9b9b9b",
    linestyle="dotted",
)

In [ ]:
linesvcminus = matplotlib.lines.Line2D(
    xdata=X_1,
    ydata=-coeffs[0] / coeffs[1] * X_1 - coeffs[2] / coeffs[1] + 1,
    label="$\mathbf{w}\cdot\mathbf{x} + b = -1$",
    color="#9b9b9b",
    linestyle="dashed",
)

In [ ]:
fig, ax = plt.subplots()
plt.scatter(x=data[:20, 0], y=data[:20, 1], c="green", label="$Y = 1$")
plt.scatter(x=data[20:, 0], y=data[20:, 1], c="red", label="$Y = -1$")
plt.axis("equal")
plt.xlabel("$X_1$")
plt.ylabel("$X_2$")
plt.xlim((-0.5, 1.5))
plt.ylim((-0.5, 1.5))
sv = plt.scatter(
    x=xsupport[:, 0],
    y=xsupport[:, 1],
    marker="*",
    s=250,
    label="Support Vectors",
)
ax.add_line(linesvc)
ax.add_line(linesvcplus)
ax.add_line(linesvcminus)

plt.legend(
    bbox_to_anchor=(1, 1), handles=[linesvc, sv, linesvcplus, linesvcminus]
)
plt.show()

### Can you separate this dataset?

There are datasets for which enforcing a linear separation boundary is just a terrible idea,
regardless of how soft we let the margin be. For example:

In [ ]:
np.random.seed(42)

In [ ]:
n_samples_per_class = 100

In [ ]:
R_minus = np.random.uniform(low=0.0, high=2.0, size=(n_samples_per_class, 1))
theta_minus = np.random.uniform(
    low=0.0, high=2 * np.pi, size=(n_samples_per_class, 1)
)

In [ ]:
R_plus = np.random.uniform(low=4.0, high=6.0, size=(n_samples_per_class, 1))
theta_plus = np.random.uniform(
    low=0.0, high=2 * np.pi, size=(n_samples_per_class, 1)
)

In [ ]:
Y_plus_insep2_sample = np.concatenate(
    [R_plus * np.cos(theta_plus), 4 + R_plus * np.sin(theta_plus)], axis=1
)
Y_minus_insep2_sample = np.concatenate(
    [R_minus * np.cos(theta_minus), 4 + R_minus * np.sin(theta_minus)], axis=1
)

In [ ]:
features_insep2 = np.concatenate(
    [Y_plus_insep2_sample, Y_minus_insep2_sample], axis=0
)

labels_insep2 = np.ones(shape=(2 * n_samples_per_class, 1))
labels_insep2[n_samples_per_class:] = -1

data_insep2 = np.concatenate([features_insep2, labels_insep2], axis=1)

In [ ]:
plt.scatter(
    x=data_insep2[:n_samples_per_class, 0],
    y=data_insep2[:n_samples_per_class, 1],
    c="green",
    label="$Y = 1$",
)
plt.scatter(
    x=data_insep2[n_samples_per_class:, 0],
    y=data_insep2[n_samples_per_class:, 1],
    c="red",
    label="$Y = -1$",
)
plt.axis("equal")
plt.xlabel("$X_1$")
plt.ylabel("$X_2$")
plt.xlim((-6, 6))
plt.ylim((-2, 10))
plt.legend()
plt.show()

It's clear that a separation boundary should be circular rather than linear.
And yet, SVMs can still be used to separate this dataset. The trick is to
map the data to a typically higher dimensional space first, where it is linearly separable,
and apply the SVM algorithm in that space. As an example,
since we strongly suspect from the picture (or our generation parameters...)
that a boundary of the form
$$X_1^2 + (X_2 - c_2)^2 = r^2$$
(which is the equation of a circle centered on $(0, c_2)$ with radius $r$) should be able to separate the data, let's try the map
$$\Phi: (X_1, X_2) \longrightarrow (X_1^2, X_2, X_2^2).$$
This is what the data looks like after applying $\Phi$ to the elements of our dataset.

In [ ]:
# We'll need the following package to make 3D plots with matplotlib
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Let's compute the 3D features, i.e. the map Phi

data_insep2_highdim = np.concatenate(
    [
        (data_insep2[:, 0] ** 2).reshape(-1, 1),
        (data_insep2[:, 1]).reshape(-1, 1),
        (data_insep2[:, 1] ** 2).reshape(-1, 1),
        data_insep2[:, 2].reshape(-1, 1),
    ],
    axis=1,
)

In [ ]:
fig = plt.figure()

ax = Axes3D(fig)

ax.scatter(
    data_insep2_highdim[:, 0],
    data_insep2_highdim[:, 1],
    data_insep2_highdim[:, 2],
    c=data_insep2[:, 2].astype(int),
    cmap=matplotlib.colors.ListedColormap(colors),
)

ax.set_xlabel("$X_1^2$")
ax.set_ylabel("$X_2$")
ax.set_zlabel("$X_2^2$")
ax.view_init(azim=190)

plt.show()

That looks promising! Let's find a separating hyperplane in this higher-dimensional feature space. 

In [ ]:
C = 0.1

In [ ]:
P = np.zeros(shape=(2 * n_samples_per_class + 4, 2 * n_samples_per_class + 4))
P[0, 0] = 1
P[1, 1] = 1
P[2, 2] = 1
P = matrix(P)

In [ ]:
q = C * np.ones(shape=(2 * n_samples_per_class + 4, 1))
q[0] = 0.0
q[1] = 0.0
q[2] = 0.0
q[3] = 0.0
q = matrix(q)

In [ ]:
G = -data_insep2_highdim[:, 3].reshape((-1, 1)) * np.concatenate(
    [data_insep2_highdim[:, :3], np.ones((data_insep2_highdim.shape[0], 1))],
    axis=1,
)
G = np.concatenate([G, -np.identity(2 * n_samples_per_class)], axis=1)
G_positive_penalty = np.concatenate(
    [
        np.zeros(shape=(2 * n_samples_per_class, 4)),
        -np.identity(2 * n_samples_per_class),
    ],
    axis=1,
)
G = np.concatenate([G, G_positive_penalty], axis=0)
G = matrix(G)

In [ ]:
h = np.concatenate(
    [
        -np.ones(shape=(data_insep2_highdim.shape[0], 1)),
        np.zeros(shape=(data_insep2_highdim.shape[0], 1)),
    ],
    axis=0,
)
h = matrix(h)

In [ ]:
solution = qp(P, q, G, h)

In [ ]:
coeffs = np.array(solution["x"])

In [ ]:
coeffs[:4]

The hyperplane separting hyperplane found by the `cvxopt` package
has equation $0.166X_1^2 - 1.7X_2 + 0.135X_2^2 + 0.633 = 0$.
As a sanity check, let us plot this decision boundary back in
2D space. 

In [ ]:
delta = 0.05
x1_cont = np.arange(-6, 6, delta)
x2_cont = np.arange(-2, 10, delta)
X1_cont, X2_cont = np.meshgrid(x1_cont, x2_cont)

In [ ]:
Z = (
    coeffs[0] * X1_cont ** 2
    + coeffs[1] * X2_cont
    + coeffs[2] * X2_cont ** 2
    + coeffs[3]
)

In [ ]:
fig, ax = plt.subplots()
plt.scatter(
    x=data_insep2[:n_samples_per_class, 0],
    y=data_insep2[:n_samples_per_class, 1],
    c="green",
    label="$Y = 1$",
)
plt.scatter(
    x=data_insep2[n_samples_per_class:, 0],
    y=data_insep2[n_samples_per_class:, 1],
    c="red",
    label="$Y = -1$",
)
plt.axis("equal")
plt.xlabel("$X_1$")
plt.ylabel("$X_2$")
linesvc = plt.contour(X1_cont, X2_cont, Z, levels=[0], colors=("#e88127"))
linesvc.collections[0].set_label(
    "$0.166X_1^2 - 1.7X_2 + 0.135X_2^2 + 0.633 = 0$"
)
plt.legend(bbox_to_anchor=(1, 1))
plt.show()

Not bad!

### The Lagrangian Dual

We will quickly run into computational problems if our proposed map $\Phi$
maps the data to a very high dimensional space. In fact, we could even attempt
to map the data to a space with an arbitrary number of dimensions, and then it's
not clear at all that we can formulate the optimization problem in a tractable way.
Thankfully, there is one last trick that will make this possible: to consider
the *dual* optimization problem.

Given an optimization problem of the form

$$\text{Minimize }f(\mathbf{x})\text{ subject to }g_i(\mathbf{x}) \geq 0 \text{ for } 1\leq i \leq n,$$

let's introduce *Lagrange multipliers* $\lambda_i \geq 0$, one for each constraint, and define the
*Lagrangian* to be 
$$L(\mathbf{\lambda})  = \inf_\mathbf{x} \left( f(\mathbf{x}) - \sum_{i=1}^n \lambda_i g_i(\mathbf{x})\right)$$

(if you don't know what $\inf$ means, just think $\min$). The *Lagrangian dual problem* is then defined to be

$$\text{Maximize }L(\mathbf{\lambda})\text{ subject to }\lambda_i \geq 0 \text{ for } 1\leq i \leq n.$$

In this context, the original optimization problem is called the *primal* problem.
When the optimization problem is convex (and some technical condititons are satisfied), as is the case of support vector machines, it turns out
that these two problems are equivalent. In other words, one could solve the dual first, giving
an optimal value for $\mathbf{\lambda}$, call it $\mathbf{\lambda}^*$; the optimal value of $\mathbf{x}$ is then that which
minimizes $f(\mathbf{x}) - \sum_{i=1}^n \lambda^*_i g_i(\mathbf{x})$. We won't prove here why this holds,
but let's get an intuition for what is going on by considering a simple example. Consider the following simple
optimization problem: minimize
$$f(X_1, X_2) = X_1^2 + X_2^2$$
subject to
$$g(X_1, X_2) \geq 0$$
where
$$g(X_1, X_2) = X_2 + X_1 - 1.$$
There is only one constraint here, so we will introduce only one Lagrange multiplier $\lambda$.
To find the value of the Lagrangian for a fixed value of $\lambda \geq 0$, we must
minimize $X_1^2 + X_2^2 - \lambda(X_2 + X_1 - 1)$ with respect to $X_1$ and $X_2$. That's resonably easy
(we are minimizing a second order degree polynomials in $X_1$ and minimizing a second order degree polynomials in $X_2$
separately). The optimal values for $X_1$ is $\lambda/2$ and likewise for $X_2$. We can therefore compute the value
of the Lagrangian: $L(\lambda) = \lambda - \lambda^2/2$. This is maximized for $\lambda = 1$ and so we deduce that 
$X_1^2 + X_2^2$ is minimized when $X_1 = X_2 = 1/2$ with minimal value $1/2$.

In [ ]:
g = np.linspace(start=-6, stop=6, num=100)

In [ ]:
f = (g + 1) ** 2 / 2

In [ ]:
top = np.array([10 for i in range(100)])

In [ ]:
def lagrangian(l):
    return l - l ** 2 / 2


def lagrangian_line(g, l):
    return l * g + lagrangian(l)

In [ ]:
lambda2 = matplotlib.lines.Line2D(
    xdata=g,
    ydata=lagrangian_line(g, 2),
    label="$f - 2g = L(2)$",
    color="#e88127",
    linestyle="dashed",
)

In [ ]:
lambda1 = matplotlib.lines.Line2D(
    xdata=g,
    ydata=lagrangian_line(g, 1),
    label="$f - g = L(1)$",
    color="#e88127",
    linestyle="solid",
)

In [ ]:
lambda3 = matplotlib.lines.Line2D(
    xdata=g,
    ydata=lagrangian_line(g, 3),
    label="$f - 3g = L(3)$",
    color="#e88127",
    linestyle="dotted",
)

In [ ]:
fig, ax = plt.subplots()
ax.fill_between(
    g, f, top, where=top >= f, facecolor="#9b9b9b", interpolate=True
)

plt.scatter(x=[0, 0, 0], y=[lagrangian(1), lagrangian(2), lagrangian(3)])


plt.annotate("$L(1)$", xy=(0, lagrangian(1)))
plt.annotate("$L(2)$", xy=(0, lagrangian(2)))
plt.annotate("$L(3)$", xy=(0, lagrangian(3)))

ax.axvline(x=0, color="k")


ax.add_line(lambda2)
ax.add_line(lambda1)
ax.add_line(lambda3)

plt.axis("equal")
plt.xlim((-5, 5))
plt.ylim((-2, 5))
plt.legend(bbox_to_anchor=(1, 1), handles=[lambda1, lambda2, lambda3])

plt.xlabel("$g(X_1, X_2)$")
plt.ylabel("$f(X_1, X_2)$")

plt.show()

In the figure above, we have shaded the set of possible values that $f$ and $g$ can take simultaneously
in grey. We have also drawn the lines $f - \lambda g = L(\lambda)$ for $\lambda \in \{1, 2, 3\}$.
As you can see, the intercepts of these lines provide lower bounds on the minimal value that $f$ can take
subject to $g \geq 0$. Moreover, these lower bounds converge to the minimal value of $f$ as $\lambda$ tends to $1$.

Question: What does it mean for a set to be convex? Can you see why the intercepts need not converge to the
minimal value of $f$ if the shaded area is not convex?

### Applying the dual to Support Vector Machines

We leave it as an exercise to the reader to show that the dual formulation of the
Suppport Vector Machine, Soft Margin optimization problem when $k=1$ is


**Suport Vector Machine, Soft Margin, Dual Form**: Maximize

$$\sum_{i=1}^n \lambda_i - \frac{1}{2} \sum_{i, j} y_iy_j\lambda_i\lambda_j \mathbf{x}_i \cdot \mathbf{x}_j$$
subject to:
$$0\leq \lambda_i\leq C$$
and
$$\sum_{i=1}^n \lambda_i y_i = 0$$

The optimal value of $\mathbf{w}$ is then given by 
$\mathbf{w} = \sum_{i=1}^n \lambda_iy_i\mathbf{x}_i$ and $b = (1-y_i\mathbf{x}_i\cdot \mathbf{w})/y_i$ for any $i$ such that $0 < \lambda_i < C$.
Practically, you should take the average over all such values to get a
more numerically stable result.

The dual formulation of the SVM algorithm has two key features:
* The constraints are easier to deal with.
* The feature space appears implicitly rather than explicitly:
all that one needs to know about the feature space is the
value of the scalar product between data vectors.

The latter feature is very important. In primal form we would 
have had to optimize explicitly over the coordinates of $\mathbf{w}$,
which may be computationally very hard or impossible to do. 

### Kernels

We saw in the previous section how the dual formulation of the SVM optimization problem only required us to know the scalar product between data vectors. This allows us to consider maps $\Phi$ mapping the data to a feature space of arbitarily large dimension before running the SVM algorithm, and we do not need to know $\Phi$ explicitly. Rather, what we care about is the *kernel* of $\Phi$ which is defined by
$$K(\mathbf{x}, \mathbf{y}) = \Phi(\mathbf{x}) \cdot \Phi(\mathbf{y}),$$
and is in practice what we have to pass to an implementation of the SVM algorithm. 

One of the most natural kernels to use for training an SVM is the *polynomial* kernel.
The polynomial kernel is defined by
$$K(\mathbf{x}, \mathbf{y}) = (\mathbf{x}\cdot  \mathbf{y} + c)^d$$
where $d$ and $c$ are constants. Notice how for $d=2$ this naturally generalizes the approach
we took for classifying the circular dataset.

Another popular kernel is the Radial Basis Function (RBF) kernel. It is defined by
$$K(\mathbf{x}, \mathbf{y}) = \exp\left(-\frac{\|\mathbf{x} - \mathbf{y} \|}{2\sigma^2}\right).$$
You should think of the RBF kernel as a similarity measure between data points
where vectors which are close together with respect to the $l^2$-norm are similar
(Notice how $0 \leq K(\mathbf{x}, \mathbf{y}) \leq 1$ and $K(\mathbf{x}, \mathbf{y}) \to 1$ as
$\|\mathbf{x} - \mathbf{y} \| \to 0$). The hyperparameter $\sigma$ controls
the distribution of similarity as a function of distance. When you select
a small value for $\sigma$, only data points which are very close together are deemed to be similar.
You should then expect to find a separating boundary which is 
more sensitive to the local topology of your dataset, and hence you could risk overfitting.
When you select a large value of $\sigma$, there is less of a difference in similarity
between closer and farther data points. Your separating boundary will then be more influenced
by long-range topological features, and hence you could risk underfitting. As usual,
you will have to select $\sigma$ through cross-validation! 

Let us revisit our "circular" dataset. First we'll train the same SVM as before
using the dual formulation of the optimization problem. This amounts to defining
a custom kernel. Then we'll try an RBF kernel as well.

In [ ]:
P = matrix(
    np.matmul(
        data_insep2_highdim[:, 3].reshape((-1, 1))
        * data_insep2_highdim[:, :3],
        (
            data_insep2_highdim[:, 3].reshape((-1, 1))
            * data_insep2_highdim[:, :3]
        ).T,
    )
)

In [ ]:
q = matrix(-np.ones(shape=(2 * n_samples_per_class, 1)))

In [ ]:
G = matrix(
    np.concatenate(
        [
            -np.identity(2 * n_samples_per_class),
            np.identity(2 * n_samples_per_class),
        ],
        axis=0,
    )
)
h = matrix(
    np.concatenate(
        [
            np.zeros(shape=(2 * n_samples_per_class, 1)),
            C * np.ones(shape=(2 * n_samples_per_class, 1)),
        ],
        axis=0,
    )
)
A = matrix(data_insep2_highdim[:, 3].reshape((1, -1)))
b = matrix(np.zeros(shape=(1, 1)))

In [ ]:
solution = qp(P, q, G, h, A, b)

In [ ]:
w = np.sum(
    np.array(solution["x"])
    * data_insep2_highdim[:, 3].reshape((-1, 1))
    * data_insep2_highdim[:, :3],
    axis=0,
).reshape((-1, 1))

In [ ]:
select = (np.array(solution["x"]) > 0.001) & (
    np.abs(np.array(solution["x"])) < 0.1 - 0.001
)

In [ ]:
Z = (
    -data_insep2_highdim[:, 3].reshape((-1, 1))
    * (data_insep2_highdim[:, :3] @ w)
    + 1
) / data_insep2_highdim[:, 3].reshape((-1, 1))

In [ ]:
Z[select]

In [ ]:
b = np.mean(Z[select])

In [ ]:
w

In [ ]:
b

We obtain the plane with equation $0.17X_1^2 - 1.08 X_2 + 0.13 X_2^2 + 0.68 = 0$,
and everything checks out :) Now let's use an RBF kernel instead.

In [ ]:
sigma = 1

In [ ]:
def K(x, y, sigma):
    return np.exp(-np.sum((x - y) ** 2, axis=1) / (2 * sigma))

In [ ]:
P = np.zeros(shape=(2 * n_samples_per_class, 2 * n_samples_per_class))
for i in range(2 * n_samples_per_class):
    for j in range(2 * n_samples_per_class):
        P[i, j] = K(
            data_insep2[i, :2].reshape((1, -1)),
            data_insep2[j, :2].reshape((1, -1)),
            sigma,
        )
P = matrix(P)

In [ ]:
q = matrix(-np.ones(shape=(2 * n_samples_per_class, 1)))

In [ ]:
G = matrix(
    np.concatenate(
        [
            -np.identity(2 * n_samples_per_class),
            np.identity(2 * n_samples_per_class),
        ],
        axis=0,
    )
)
h = matrix(
    np.concatenate(
        [
            np.zeros(shape=(2 * n_samples_per_class, 1)),
            C * np.ones(shape=(2 * n_samples_per_class, 1)),
        ],
        axis=0,
    )
)
A = matrix(data_insep2[:, 2].reshape((1, -1)))
b = matrix(np.zeros(shape=(1, 1)))

In [ ]:
solution = qp(P, q, G, h, A, b)

In [ ]:
# The following code is not very optimized :)

In [ ]:
x_test = np.array([[1, 1]])

In [ ]:
X = np.concatenate([x_test for i in range(2 * n_samples_per_class)], axis=0)

In [ ]:
select = (np.array(solution["x"]) > 0.001) & (
    np.array(solution["x"]) < C - 0.001
)

In [ ]:
b = []
for i in range(2 * n_samples_per_class):
    if select[i]:
        x = np.repeat(
            data_insep2[i, :2].reshape(1, -1),
            repeats=2 * n_samples_per_class,
            axis=0,
        )
        b.append(
            (
                -data_insep2[i, 2]
                * np.sum(
                    np.array(solution["x"]).reshape(-1)
                    * data_insep2[:, 2]
                    * K(data_insep2[:, :2], x, sigma)
                )
                + 1
            )
            / data_insep2[i, 2]
        )

In [ ]:
b = np.mean(b)

In [ ]:
def f(x):
    x = np.repeat(x.reshape((1, -1)), repeats=2 * n_samples_per_class, axis=0)
    return (
        np.sum(
            K(data_insep2[:, :2], x, sigma=sigma)
            * data_insep2[:, 2]
            * np.array(solution["x"]).reshape(-1)
        )
        + b
    )

In [ ]:
delta = 0.05
x1_cont = np.arange(-6, 6, delta)
x2_cont = np.arange(-2, 10, delta)
X1_cont, X2_cont = np.meshgrid(x1_cont, x2_cont)
Z = np.zeros(shape=(240, 240))

In [ ]:
for i in range(240):
    for j in range(240):
        Z[i, j] = f(np.array([X1_cont[i, j], X2_cont[i, j]]))

In [ ]:
fig, ax = plt.subplots()
plt.scatter(
    x=data_insep2[:n_samples_per_class, 0],
    y=data_insep2[:n_samples_per_class, 1],
    c="green",
    label="$Y = 1$",
)
plt.scatter(
    x=data_insep2[n_samples_per_class:, 0],
    y=data_insep2[n_samples_per_class:, 1],
    c="red",
    label="$Y = -1$",
)
plt.axis("equal")
plt.xlabel("$X_1$")
plt.ylabel("$X_2$")
linesvc = plt.contour(X1_cont, X2_cont, Z, levels=[0], colors=("#e88127"))
linesvc.collections[0].set_label("RBF boundary")
plt.legend(bbox_to_anchor=(1, 1))
plt.show()

Exercise: Investigate the effect of $\sigma$ by varying its value. What do you notice?

### Acknowledgements
This tutorial is largely based on the material found in https://www.di.ens.fr/~mallat/papiers/svmtutorial.pdf.
Many thanks to Ben G, Laurence and Milica for reviewing an earlier version of this post. Credits to Laurence for beautiful `fig1` and `fig2`.